In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import plotly.express as exp

data = pd.read_csv('BX-Users.csv')


#---katharismos tou column location gia na meinei mono h xwra---
for x in range(len(data)):
    my_str = data.loc[x,"location"]
    new_val= my_str[my_str.rfind(','):]
    data.loc[x,"location"] = new_val[1:len(new_val)]
    # clear people above 110 age
    if data.loc[x,"age"] >110:
        data.drop(x,axis=0,inplace=True)

#---katharismos twn countries kathws uphrxan mesa mi uparktes xwres.---
unique_countries = data['location'].unique()
#arxikos arithmos xwrwn
print("Xwres prin to katharisma: ", len(unique_countries))

#---diagrafi twn xwrwn pou emfanizontai ligoteres apo 15 fores kathws opws eidame den einai egkyres
for country in unique_countries:  
    if len(data[data['location'] == country]) < 15 :
        #print(country, "with entries ", len(data[data['location'] == country]), "DROPPED")
        data.drop(data.loc[data['location'] == country].index, inplace = True) 

unique_countries = data['location'].unique()
print("Xwres meta to katharisma: ", len(unique_countries))

#function gia na plottarei tis meses hlikies kathe xwras
def makePlot(number): 

    for x in range(number):
        country = unique_countries[x]
        age18_25 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 25) & (data[data['location'] == country]['age'] >= 18)]
        age26_35 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 35) & (data[data['location'] == country]['age'] >= 26)]
        age36_45 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 45) & (data[data['location'] == country]['age'] >= 36)]
        age46_55 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 55) & (data[data['location'] == country]['age'] >= 46)]
        age55above = data[data['location'] == country]['age'][data[data['location'] == country]['age'] >= 56]

        x = ["18-25","26-35","36-45","46-55","55+"]
        y = [len(age18_25.values),len(age26_35.values),len(age36_45.values),len(age46_55.values),len(age55above.values)]

        plt.figure(figsize=(15,6))
        sns.barplot(x=x, y=y, palette="rocket")
        plt.title(country)
        plt.xlabel("Age")
        plt.ylabel("COUNT OF PEOPLE")
        plt.show()  

#makePlot(5)

              





,uid,location,age
0,1,usa,NaN
1,2,usa,18.0
2,3,russia,NaN
3,4,portugal,17.0
4,5,united kingdom,NaN


In [52]:
# pairnoume mono ta rows pou aforoun mia xwra kai dropparoume tis Nan hlikies
# auto meta tha ginei gia oles tis xwres mia mia
each_country = data.loc[data['location'] == ' usa']
each_country = each_country.dropna()

#metatropi tis xwras se 1 se ola ta rows gia na mporei na mpei sto montelo
each_country['location'] = each_country['location'].replace(each_country.iloc[0]['location'], 1)



# #######$$$$$$$$$$@@@@@@@@@@@@ .   apo edw kai pera poy exw kanei kai tis xwres 1 prepei na dw pws tha mpei auto sto montelo


#edw ginetai kanonikopoisi -- den ksero ama xreiazetai sigoura
scaler = StandardScaler()
scaled_features = scaler.fit_transform(each_country)

scaled_features



array([[-1.73681248,  0.        , -1.33568078],
       [-1.73676253,  0.        ,  1.55336004],
       [-1.73658773,  0.        , -1.26849379],
       ...,
       [ 1.74463113,  0.        , -0.66381082],
       [ 1.7446561 ,  0.        ,  1.62054704],
       [ 1.74475599,  0.        ,  0.47836811]])

In [55]:
kmeans = KMeans(
    init="random",
    n_clusters=3,
    n_init=10,
    max_iter=300,
    random_state=42
)

model = kmeans.fit(scaled_features)

print(model.labels_)



[0 1 0 ... 2 1 2]


InvalidIndexError: (slice(None, None, None), 0)

<Figure size 800x600 with 0 Axes>